In [21]:
import pandas as pd
import numpy as np
import praw
from praw.models import MoreComments
import os
from groq import Groq

In [20]:
!pip install groq

  Using cached groq-0.18.0-py3-none-any.whl.metadata (14 kB)
Using cached groq-0.18.0-py3-none-any.whl (121 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [50]:
reddit = praw.Reddit(user_agent="Extract Reddit v1.0 by u/External_Middle8912",
                     client_id="KtFyZUw2ZS0gPLVwnWpITw", 
                     client_secret="8xdnOL6gEBbwCgu389aJrj3c1hKj7w")
# url = "https://www.reddit.com/r/depression/comments/1iswzg5/i_forget_depression_isnt_common/"
category = ["depression", "anxiety", 'mentalillness']
data = []
for item in category:
    subreddit = reddit.subreddit(item)
    for submission in subreddit.hot(limit=1000):
        data.append([
            submission.title,       # Post Title
            submission.selftext,
            submission.score,       # Upvote Score
            submission.num_comments,# Number of Comments
            submission.url,         # Post URL
            submission.author,      # Author Name
            submission.created_utc,  # Timestamp (Unix)
            item
        ])
posts = pd.DataFrame(data,columns=["Title", "Text", "Score", "Comments", "URL", "Author", "Created_UTC", "Subreddit"])       
# indexNames = posts[(posts.body == '[removed]') | (posts.body == '[deleted]')].index
# posts.drop(indexNames, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'body'

In [51]:
posts['Text'] = posts['Text'].str.replace(r'\\r\\n|\\n|\\', " ", regex = True)

In [52]:
posts['Text'].iloc[5]

'There has been almost a week that I have no desire to do anything than sleep.The only time I leave my bed is to go to the bathroom.I barely eat or drink any water.When I wake up I immediately want to go to sleep again.Most of the times I end up crying myself to sleep wishing to never wake up again.This is the worst time of my life I can\'t express how much distress I feel, sometimes with no reason at all I am extremely anxious,I have lost interest at all my hobbies.I have skipped some very important college classes and it doesn\'t seem to be getting better.\n\nI don\'t think anyone cares about me in my life and I am starting to believe I will never be loved.I tried to reach out at one of my classmates which I hang out some times but it didn\'t seem he cared that much because I got no reaction.And I don\'t blame him why would he?We are not even that close.I am honestly incapable of human communication.\n\nMy family doesn\'t care either because they haven\'t questioned the fact that I h

In [53]:
posts

,Title,Text,Score,Comments,URL,Author,Created_UTC,Subreddit
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,2366,177,https://www.reddit.com/r/depression/comments/d...,SQLwitch,1.572361e+09,depression
1,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,26,247,https://www.reddit.com/r/depression/comments/1...,SQLwitch,1.727565e+09,depression
2,i regret ever opening up to anyone,they make you feel even shittier about what yo...,53,20,https://www.reddit.com/r/depression/comments/1...,Acrobatic_Lie7392,1.740252e+09,depression
3,I'm going to commit suicide in a few days,I’m 19 years old and I just can't live like th...,41,37,https://www.reddit.com/r/depression/comments/1...,Ewexz,1.740245e+09,depression
4,"I hate happy families, especially dad-daughter...",I feel so disappointed and sad whenever I see ...,36,6,https://www.reddit.com/r/depression/comments/1...,plushielvr,1.740246e+09,depression
...,...,...,...,...,...,...,...,...
2797,Obsession with deceased celebrity?,"This is not a joke, please take it seriously.\...",10,6,https://www.reddit.com/r/mentalillness/comment...,tikhung01,1.736532e+09,mentalillness
2798,Please help! Why am I all of a sudden angry at...,So yesterday I was haveing a good day then I R...,3,4,https://www.reddit.com/r/mentalillness/comment...,Bloody_skulls_,1.736555e+09,mentalillness
2799,I had to call crisis on my manic boyfriend and...,My boyfriend has had something mentally going ...,2,30,https://www.reddit.com/r/mentalillness/comment...,Few-Recover-4973,1.736563e+09,mentalillness
2800,Do you think it’s too late to tell my psychiat...,I’ve been seeing her for about a year & a half...,3,2,https://www.reddit.com/r/mentalillness/comment...,Mental-Net1389,1.736554e+09,mentalillness


In [75]:
groq_API = "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"
client = Groq(api_key=groq_API)

def categorize_reddit_content(content):
      messages = [
          {
              "role": "system",
              "content": 'null'
          },
          {
              "role": "user",
              "content": f"Please categorize our reddit post content into one of the three categories: 'Depression', 'Anxiety', 'Mental Illness': \n\n {content}. The output should only be a word with no extra texts."
          }
      ]

      # prompt = "Please categorize our reddit post content into one of the three categories: 'Depression', 'Anxiety', 'Mental Illness'."

      chat_completion = client.chat.completions.create(
              temperature = 0.15, 
              messages = messages, 
              model = "llama-3.3-70b-versatile", 
              n = 1, 
      )
      category = chat_completion.choices[0].message.content
      return category

# print(chat_completion.choices[0].message.content)

In [63]:
type(sub_df['Text'])

pandas.core.series.Series

In [76]:
sub_df = posts[:5]
sub_df["New_Text"] = sub_df["Text"].apply(categorize_reddit_content)


/var/folders/hk/1l4v0gdd41z7xrwc5_b0pcmm0000gn/T/ipykernel_64497/2596895960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["New_Text"] = sub_df["Text"].apply(categorize_reddit_content)


In [77]:
sub_df['New_Text']

0    Depression
1    Depression
2    Depression
3    Depression
4    Depression
Name: New_Text, dtype: object

In [78]:
posts.to_csv("/Users/bowiechuang/Documents/GitHub/baychuang/reddit_post.csv")